# Exercise 4.1

In the three-period model of Figure 1.2.2 of Chapter 1, let the interest rate be $r=\frac{1}{4}$ so the risk-neutral probabilities are $\tilde{p}=\tilde{q}=\frac{1}{2}$.

(i) Determine the price at time zero, denoted $V_0^P$, of the American put that expires at time three and has intrinsic value $g_P(s)=(4-s)^+$.

(ii) Determine the price at time zero, denoted $V_0^C$, of the American call that expires at time three and has intrinsic value $g_C(s)=(s-4)^+$.

(iii) Determine the price at time zero, denoted $V_0^S$, of the American straddle that expires at time three and has intrinsic value $g_S(s)=g_P(s)+g_C(s)$.

(iv) Explain why $V_0^S<V_0^P+V_0^C$.

# Answer

(i) Note that we can use the pricing algorithm (4.2.5),(4.2.6) for non-path-dependent american derivatives.  In other words, we can price the security with the algorithm
\begin{align}
&v_N(s)=\max\{g(s),0\},\\
&v_n=\max\{g(s), \frac{1}{1+r}[\tilde{p}v_{n+1}(us)+\tilde{q}v_{n+1}(ds)]\}, \; n=N-1,\dots,0.
\end{align}
To avoid doing the numerious calculation above by hand, we write a Python code to perform these calculations for us.

In [1]:
# returns list of possible sequence of coin tosses of length n.
def possible_coin_tosses(n=1):
    coin_toss_possibilities = []
    for bin_seq in [(n-len(bin(i)[2:]))*'0'+bin(i)[2:] for i in range(2**n)]:
        coin_toss_seq = ''
        for bit in bin_seq:
            if bit == '0':
                coin_toss_seq = coin_toss_seq + 'H'
            elif bit == '1':
                coin_toss_seq = coin_toss_seq + 'T'
        coin_toss_possibilities.append(coin_toss_seq)
    return coin_toss_possibilities
print(possible_coin_tosses(n=2))

['HH', 'HT', 'TH', 'TT']


In [2]:
# based on constant d, r, and u factors, generate binomial model
# list of stock prices, given some initial stock price,
# from time 0 to time t_f, inclusive
def bm_stock_prices(s_0=4, d=0.5, r=0.25, u=2, t_f=3):
    S={0:{'': s_0}}
    for t in range(1,t_f+1):
        S[t]={}
        for coin_toss_seq in possible_coin_tosses(t):
            mult_factor = 1
            for toss in coin_toss_seq:
                if toss == 'H':
                    mult_factor *= u
                elif toss == 'T':
                    mult_factor *= d
            S[t][coin_toss_seq]=s_0*mult_factor
    return S
S = bm_stock_prices()
print(S)

{0: {'': 4}, 1: {'H': 8, 'T': 2.0}, 2: {'HH': 16, 'HT': 4.0, 'TH': 4.0, 'TT': 1.0}, 3: {'HHH': 32, 'HHT': 8.0, 'HTH': 8.0, 'HTT': 2.0, 'THH': 8.0, 'THT': 2.0, 'TTH': 2.0, 'TTT': 0.5}}


In [3]:
# compute intrinsic value of american put with strike price K,
# and underlying stock price s
def american_put_g(s, K=4):
    return max([K-s, 0])
# generate dictionary of intrinsic values for all possible coin tosses from time 0
# to time t_f, inclusive, based on the stock prices in the dictionary S
Gp = {}
for t in S.keys():
    Gp[t]={}
    for coin_toss_seq, stock_price in S[t].items():
        Gp[t][coin_toss_seq] = american_put_g(s=stock_price, K=4)
print(Gp)

{0: {'': 0}, 1: {'H': 0, 'T': 2.0}, 2: {'HH': 0, 'HT': 0.0, 'TH': 0.0, 'TT': 3.0}, 3: {'HHH': 0, 'HHT': 0, 'HTH': 0, 'HTT': 2.0, 'THH': 0, 'THT': 2.0, 'TTH': 2.0, 'TTT': 3.5}}


In [4]:
# up factor, down factor, interest rate, and corresponding risk-neutral probabilities
d = 0.5
u = 2
r = 0.25
p = (1+r-d)/(u-d)
q = (u-1-r)/(u-d)

In [5]:
# Based on the stock prices in the dictionary S and the intrinsic values in
# the dictionary G, calculate the derivative prices V
Vp = {}
for t in list(S.keys())[::-1]:
    Vp[t] = {}
    for coin_toss_seq, value in Gp[t].items():
        if t==max(S.keys()):
            Vp[t][coin_toss_seq] = max([value, 0])
        else:
            rnd_price = (1/(1+r))*(p*Vp[t+1][coin_toss_seq+'H']+q*Vp[t+1][coin_toss_seq+'T'])
            Vp[t][coin_toss_seq] = max([value, rnd_price])     
print(Vp) 

{3: {'HHH': 0, 'HHT': 0, 'HTH': 0, 'HTT': 2.0, 'THH': 0, 'THT': 2.0, 'TTH': 2.0, 'TTT': 3.5}, 2: {'HH': 0, 'HT': 0.8, 'TH': 0.8, 'TT': 3.0}, 1: {'H': 0.32000000000000006, 'T': 2.0}, 0: {'': 0.9280000000000002}}


As we can see from the dictionary above, $V_0^P=$ `V[0]['']` $=0.928. \square$

(ii) We can take the same approach as we did for (i).  We just have to compute different dictionary for $G$ because the intrinsic value function has changed. 

In [6]:
# compute intrinsic value of american call with strike price K,
# and underlying stock price s
def american_call_g(s, K=4):
    return max([s-K, 0])
# generate dictionary of intrinsic values for all possible coin tosses from time 0
# to time t_f, inclusive, based on the stock prices in the dictionary S
Gc = {}
for t in S.keys():
    Gc[t]={}
    for coin_toss_seq, stock_price in S[t].items():
        Gc[t][coin_toss_seq] = american_call_g(s=stock_price, K=4)
print(Gc)

{0: {'': 0}, 1: {'H': 4, 'T': 0}, 2: {'HH': 12, 'HT': 0.0, 'TH': 0.0, 'TT': 0}, 3: {'HHH': 28, 'HHT': 4.0, 'HTH': 4.0, 'HTT': 0, 'THH': 4.0, 'THT': 0, 'TTH': 0, 'TTT': 0}}


In [7]:
# Based on the stock prices in the dictionary S and the intrinsic values in
# the dictionary G, calculate the derivative prices V
Vc = {}
for t in list(S.keys())[::-1]:
    Vc[t] = {}
    for coin_toss_seq, value in Gc[t].items():
        if t==max(S.keys()):
            Vc[t][coin_toss_seq] = max([value, 0])
        else:
            rnd_price = (1/(1+r))*(p*Vc[t+1][coin_toss_seq+'H']+q*Vc[t+1][coin_toss_seq+'T'])
            Vc[t][coin_toss_seq] = max([value, rnd_price])     
print(Vc)

{3: {'HHH': 28, 'HHT': 4.0, 'HTH': 4.0, 'HTT': 0, 'THH': 4.0, 'THT': 0, 'TTH': 0, 'TTT': 0}, 2: {'HH': 12.8, 'HT': 1.6, 'TH': 1.6, 'TT': 0}, 1: {'H': 5.760000000000001, 'T': 0.6400000000000001}, 0: {'': 2.5600000000000005}}


As we can see from the dictionary above, $V_0^C=$ `V[0]['']` $=2.56. \square$

(iii) Again we can take the same approach as we did for (i)-(ii), but we first have to compute different dictionary for $G$ because the intrinsic value function has changed.

In [8]:
# compute intrinsic value of american call with strike price K,
# and underlying stock price s
def american_straddle_g(s, K=4):
    return american_put_g(s=s, K=K) + american_call_g(s=s, K=K)
# generate dictionary of intrinsic values for all possible coin tosses from time 0
# to time t_f, inclusive, based on the stock prices in the dictionary S
Gs = {}
for t in S.keys():
    Gs[t]={}
    for coin_toss_seq, stock_price in S[t].items():
        Gs[t][coin_toss_seq] = american_straddle_g(s=stock_price, K=4)
print(Gs)

{0: {'': 0}, 1: {'H': 4, 'T': 2.0}, 2: {'HH': 12, 'HT': 0.0, 'TH': 0.0, 'TT': 3.0}, 3: {'HHH': 28, 'HHT': 4.0, 'HTH': 4.0, 'HTT': 2.0, 'THH': 4.0, 'THT': 2.0, 'TTH': 2.0, 'TTT': 3.5}}


In [9]:
# Based on the stock prices in the dictionary S and the intrinsic values in
# the dictionary G, calculate the derivative prices V
Vs = {}
for t in list(S.keys())[::-1]:
    Vs[t] = {}
    for coin_toss_seq, value in Gs[t].items():
        if t==max(S.keys()):
            Vs[t][coin_toss_seq] = max([value, 0])
        else:
            rnd_price = (1/(1+r))*(p*Vs[t+1][coin_toss_seq+'H']+q*Vs[t+1][coin_toss_seq+'T'])
            Vs[t][coin_toss_seq] = max([value, rnd_price])     
print(Vs)

{3: {'HHH': 28, 'HHT': 4.0, 'HTH': 4.0, 'HTT': 2.0, 'THH': 4.0, 'THT': 2.0, 'TTH': 2.0, 'TTT': 3.5}, 2: {'HH': 12.8, 'HT': 2.4000000000000004, 'TH': 2.4000000000000004, 'TT': 3.0}, 1: {'H': 6.080000000000001, 'T': 2.16}, 0: {'': 3.296000000000001}}


As we can see from the dictionary above, $V_0^S=$ `V[0]['']` $=3.296. \square$

(iv) We start by proving that $V_N^S=V_N^P+V_N^C$, and that $V_n^S\leq V_n^P+V_n^C$ for $n=N-1,\dots,0$.

We start by proving the induction hypothesis.  Recall that $V_N^S(s)=\max\{g_S(s),0\}=\max\{g_C(s)+g_P(s),0\}=\max\{(s-K)^++(K-s)^+,0\}=(s-K)^++(K-s)^+=g_C(s)+g_P(s)$.

Similarly, $V_N^P(s)=\max\{g_P(s),0\}=\max\{(K-s)^+,0\}=(K-s)^+=g_P(s)$.

Similarly, $V_N^C(s)=\max\{g_C(s),0\}=\max\{(s-K)^+,0\}=(s-K)^+=g_C(s)$.

In short, $V_N^S(s)=V_N^P(s)+V_N^C(s)$.  In particular, our induction hypothesis $V_N^S(s)\leq V_N^P(s)+V_N^C(s)$ holds.

Now let $n<N$.  We want to show that if $V_{n+1}^S\leq V_{n+1}^P+V_{n+1}^C$ holds, then $V_{n}^S\leq V_{n}^P+V_{n}^C$ holds.

Recall that
\begin{align}
V_{n}^S(s)&=\max\{g_S(s), \frac{1}{1+r}[\tilde{p}V^S_{n+1}(us)+\tilde{q}V^S_{n+1}(ds)]\}\\
&=\max\{g_P(s)+g_C(s), \frac{1}{1+r}[\tilde{p}V^S_{n+1}(us)+\tilde{q}V^S_{n+1}(ds)]\}.
\end{align}
By the induction hypothesis, $V^S_{n+1}(us)\leq V^P_{n+1}(us)+V^C_{n+1}(us)$, and $V^S_{n+1}(ds)\leq V^P_{n+1}(ds)+V^C_{n+1}(ds)$.  Hence,
\begin{align}
V_{n}^S(s)&\leq \max\left\{g_P(s)+g_C(s), \frac{1}{1+r}[\tilde{p}(V^P_{n+1}(us)+V^C_{n+1}(us))+\tilde{q}(V^P_{n+1}
(ds)+V^C_{n+1}(ds))]\right\}\\
&=\max\left\{g_P(s)+g_C(s), \frac{1}{1+r}[\tilde{p}V^P_{n+1}(us)+\tilde{q}V^P_{n+1}(ds)] + \frac{1}{1+r}[\tilde{p}V^C_{n+1}(us)+\tilde{q}V^C_{n+1}(ds)]\right\}.
\end{align}

Our goal at this point is to ''break up'' the expression above according to the inequality
\begin{equation}
\max\{a+b,c+d\}\leq\max\{a,c\}+\max\{b,d\}.
\end{equation}
This inequality can be verified directly.

If $\max\{a+b,c+d\}=a+b,\;\max\{a,c\}=a,\;\max\{b,d\}=b$, then $\max\{a+b,c+d\}=\max\{a,c\}+\max\{b,d\}$.

If $\max\{a+b,c+d\}=a+b,\;\max\{a,c\}=a,\;\max\{b,d\}=d$, then $\max\{a+b,c+d\}\leq\max\{a,c\}+\max\{b,d\}$.

If $\max\{a+b,c+d\}=a+b,\;\max\{a,c\}=c,\;\max\{b,d\}=b$, then $\max\{a+b,c+d\}\leq\max\{a,c\}+\max\{b,d\}$.

Finally, if $\max\{a+b,c+d\}=a+b$, then in fact we cannot have $\max\{a,c\}=c$ and $\max\{b,d\}=d$.

Because of symmetry, we immediately arrive at the same resutls for the case $\max\{a+b,c+d\}=c+d$.

Using the inequality above and the expression for $V_{n}^S(s)$ above, we have
\begin{align}
V_{n}^S(s)&\leq\max\left\{g_P(s), \frac{1}{1+r}[\tilde{p}V^P_{n+1}(us)+\tilde{q}V^P_{n+1}(ds)]\right\} + \max\left\{g_C(s), \frac{1}{1+r}[\tilde{p}V^C_{n+1}(us)+\tilde{q}V^C_{n+1}(ds)]\right\}\\
&=V_{n}^P(s)+V_{n}^C(s).
\end{align}
This finishes the proof of our induction claim.

Note that we introduced an inequality in two places.  First, with the induction hypothesis inequalities
\begin{equation}
V^S_{n+1}(us)\leq V^P_{n+1}(us)+V^C_{n+1}(us),\\
V^S_{n+1}(ds)\leq V^P_{n+1}(ds)+V^C_{n+1}(ds).
\end{equation}
Whenever one of these inequalities is strict, we see that $V_{n+1}^S(s)<V_{n+1}^P+V_{n+1}^C(s)$.

Second, we introduced an inequality with
\begin{equation}
V_{n}^S(s)\leq\max\left\{g_P(s), \frac{1}{1+r}[\tilde{p}V^P_{n+1}(us)+\tilde{q}V^P_{n+1}(ds)]\right\} + \max\left\{g_C(s), \frac{1}{1+r}[\tilde{p}V^C_{n+1}(us)+\tilde{q}V^C_{n+1}(ds)]\right\}.
\end{equation}
This inequality is strict whenever the inequality $\max\{a+b,c+d\}\leq\max\{a,c\}+\max\{b,d\}$ is strict, where
\begin{align}
&a=g_P(s),\\
&b=g_C(s),\\
&c=\frac{1}{1+r}[\tilde{p}V^P_{n+1}(us)+\tilde{q}V^P_{n+1}(ds)],\\
&d=\frac{1}{1+r}[\tilde{p}V^C_{n+1}(us)+\tilde{q}V^C_{n+1}(ds)].
\end{align}

This is the case in the following four cases. 

Case 1: The intrinsic value of the straddle is $\geq$ than its risk-neutral, future-discounted price, $\max\{a+b,c+d\}=a+b$.  The intrinsic value of the put is $\geq$ than its risk-neutral, future-discounted price, $\max\{a,c\}=a$.  The intrinsic value of the call is $<$ than its risk-neutral, future-discounted price,  $\max\{b,d\}=d>b$.

Case 2: The intrinsic value of the straddle is $\geq$ than its risk-neutral, future-discounted price, $\max\{a+b,c+d\}=a+b$.  The intrinsic value of the put is $<$ than its risk-neutral, future-discounted price, $\max\{a,c\}=c>a$.  The intrinsic value of the call is $\geq$ than its risk-neutral, future-discounted price,  $\max\{b,d\}=b$.

Case 3: The risk-neutral, future discounted price of the straddle is $\geq$ than its intrinsic value, $\max\{a+b,c+d\}=c+d$. The risk-neutral, future discounted price of the put is $\geq$ than its intrinsic value, $\max\{a,c\}=c$. The risk-neutral, future discounted price of the put is $<$ than its intrinsic value, $\max\{b,d\}=b>d$. 

Case 4: The risk-neutral, future discounted price of the straddle is $\geq$ than its intrinsic value, $\max\{a+b,c+d\}=c+d$. The risk-neutral, future discounted price of the put is $<$ than its intrinsic value, $\max\{a,c\}=a>c$. The risk-neutral, future discounted price of the put is $\geq$ than its intrinsic value, $\max\{b,d\}=d$. 

The results above are consistent with the following intuition.  Just because it is optimal to execute a straddle, it does not mean that it is optimal to simultaneously execute a put and a call (Cases 1 and 2 above).  Similarly, just because it is better to hold off on the executing a straddle, does not mean it is optimal to simultaneously hold off on the executing of a put and a call (Cases 3 and 4 above).  We finish by remarking that the code immediately below can be used to see that we encounter Case 4 if the first coin toss results in $T$.  The code below *that* can be used to see that the induction hypothesis inequalities are strict in that case, as well as at time 0. $\square$

In [13]:
# non-induction-hypothesis inequalities
for t in range(0,2+1):
    if t==0:
        a = Gp[0]['']
        b = Gc[0]['']
        c = 1/(1+r)*(p*Vp[t+1][''+'H']+q*Vp[t+1][''+'T'])
        d = 1/(1+r)*(p*Vc[t+1][''+'H']+q*Vc[t+1][''+'T'])
        if max([a+b, c+d])==a+b and max([a, c])==a and max([b, d])>b:
            print("Case 1")
        elif max([a+b, c+d])==a+b and max([a, c])>a and max([b, d])==b:
            print("Case 2")
        elif max([a+b, c+d])==c+d and max([a, c])==c and max([b, d])>d:
            print("Case 3")
        elif max([a+b, c+d])==c+d and max([a, c])>c and max([b, d])==d:
            print("Case 4")
    elif t>0:
        for coin_toss_seq in Gp[t].keys():
            a = Gp[t][coin_toss_seq]
            b = Gc[t][coin_toss_seq]
            c = 1/(1+r)*(p*Vp[t+1][coin_toss_seq+'H']+q*Vp[t+1][coin_toss_seq+'T'])
            d = 1/(1+r)*(p*Vc[t+1][coin_toss_seq+'H']+q*Vc[t+1][coin_toss_seq+'T'])
            if max([a+b, c+d])==a+b and max([a, c])==a and max([b, d])>b:
                print("Case 1")
            elif max([a+b, c+d])==a+b and max([a, c])>a and max([b, d])==b:
                print("Case 2")
            elif max([a+b, c+d])==c+d and max([a, c])==c and max([b, d])>d:
                print("Case 3")
            elif max([a+b, c+d])==c+d and max([a, c])>c and max([b, d])==d:
                print("Time = {0}".format(t))
                print("Coin tosses = {0}".format(coin_toss_seq))
                print("Case 4")

Time = 1
Coin tosses = T
Case 4


In [14]:
# induction hypothesis inequalities
for t in range(3,0-1,-1):
    print("Time = {0}".format(t))
    print("\n")
    for coin_toss_seq in Vs[t].keys():
        flag = Vs[t][coin_toss_seq]==Vp[t][coin_toss_seq]+Vc[t][coin_toss_seq]
        print("{0}: {1}".format(coin_toss_seq, flag))
    print("\n")

Time = 3


HHH: True
HHT: True
HTH: True
HTT: True
THH: True
THT: True
TTH: True
TTT: True


Time = 2


HH: True
HT: True
TH: True
TT: True


Time = 1


H: True
T: False


Time = 0


: False


